# Bloom Filter 
### A Bloom filter is a data structure designed to tell you, rapidly and memory-efficiently, whether an element is present in a set. 
### Bloom filter does not have False Negatives but does have False Positives due to collisions.

### Data :
Given data consists of two text files: (1) Username lists which are known to be spams & (2) General Username lists.
<br>
Detect spam usernames in the (2) General usernames list 


In [0]:
!pip install mmh3

  Created wheel for mmh3: filename=mmh3-2.5.1-cp36-cp36m-linux_x86_64.whl size=37865 sha256=31d20979aaa5d3a0bfe30008289a76f649c74d2053279831a2a7e7806b3ed79d
  Stored in directory: /root/.cache/pip/wheels/38/b4/ea/6e4e321c625d3320c0c496bf4088371546d8fce5f1dd71b219
Successfully built mmh3


In [0]:
!pip install bitarray

     |████████████████████████████████| 40kB 2.2MB/s 
  Created wheel for bitarray: filename=bitarray-1.1.0-cp36-cp36m-linux_x86_64.whl size=86461 sha256=10bb3b72337d2f32b08acb934ca5568ba72ab7b22b48e938b2a7a732b1efa1ab
  Stored in directory: /root/.cache/pip/wheels/f8/e2/a6/4c8e33fba3f97ee5feff7c64341dc3abcd6aa540627e63f1b2
Successfully built bitarray


In [0]:
from google.colab import drive
drive.mount('/content/drive')
root_path = 'gdrive/My Drive/Colab Notebooks/'  #change dir to your project folder

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd 

/content/drive/My Drive/Colab Notebooks


### Reading Spam usernames

In [0]:
# Reading Spam usernames
spam_usernames = []
with open("listed_username_30.txt", "r") as f:
    for line in f:
        line = line.replace('\n','')
        spam_usernames.append(line)

### Reading Listed usernames

In [ ]:
# reading listed usernames
detect_spam =[]
with open("listed_username_365.txt", "r") as f:
    for line in f:
        line = line.replace('\n','')
        detect_spam.append(line)

### Setting up the bitarray
#### Using python wrapper for MurmurHash as a hash function and bitarray package to provide a bitarray B

In [0]:
import mmh3
import math
from bitarray import bitarray 
import pandas as pd
import numpy as np
n = 8000000
m= len(spam_usernames)
#setting up optimal k for given n and m       (defined below)
k= get_optimum_k(n,m)
# Setting BitArray
B = bitarray(n)
B.setall(0)

### Calculating optimum k


In [0]:
# optimal k calculation
def get_optimum_k(n,m):
  return (round(n/m*math.log(2)))


### Updating filter based on new entry present in the set

In [ ]:
# updating bloom filter based on new entry
def update(name,k,n,B): 
  #z=[] 
  for i in range(0,k):
    maps=mmh3.hash(name,i) % n 
   # z.append(map)
    B[maps]= True
  return B

### Function to check if the given entry is present in the set 


In [0]:
# Checking if it is a spam or not
def detect(B,word,n,k):
  detection = True
  for i in range(0,k):
    maps=mmh3.hash(word,i) % n
    if(B[maps]==False):
      detection = False
      return detection
  return detection
    

### Making the bloom filter (setting up the bit array for the given set)

In [0]:
# making the bloom filter for our example
for z in spam_usernames:
  B = update(z,k,n,B)
  


###  Counting the spam email addresses


In [0]:
PredictSpam=ActualSpam=0
start = timeit.default_timer()
for z in detect_spam:
   PredictSpam=PredictSpam+detect(B,z,n,k)
   #if z in spam_usernames:
   #  ActualSpam=ActualSpam+1
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  3.023814152000341


In [0]:
import timeit

start = timeit.default_timer()

#Your statements here

stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  1.8319999981031287e-05


In [0]:
start = timeit.default_timer()
for z in detect_spam: 
   if z in spam_usernames:
     ActualSpam=ActualSpam+1
     break;
stop = timeit.default_timer()

print('Time: ', (stop - start)*len(detect_spam)*len(spam_usernames)/(60*60))      

Time:  10512.70556504366


In [0]:
ActualSpam

106212

In [0]:
PredictSpam

1317440

In [0]:
# Percentage of false positives
def false_positive_percentage(k,m,n):
  return (1-math.exp(-k*m/n))**k


In [0]:
for i in range(0,35):
  print(i) 
  print(false_positive_percentage(i,m,n))

0
1.0
1
0.02829352443674915
2
0.0031121364104527306
3
0.0005615493649445779
4
0.00013838628291538863
5
4.2699690446096474e-05
6
1.567421758173003e-05
7
6.618195685508595e-06
8
3.1384025386340147e-06
9
1.6419363872832447e-06
10
9.347539236847525e-07
11
5.727536125995475e-07
12
3.743608901134159e-07
13
2.5909030561260956e-07
14
1.8868852092470696e-07
15
1.4383644007980983e-07
16
1.1424485321988916e-07
17
9.417231976620824e-08
18
8.028177588079652e-08
19
7.056409004404794e-08
20
6.377335043325757e-08
21
5.911887927562823e-08
22
5.609139747926816e-08
23
5.43619524465315e-08
24
5.372170808831857e-08
25
5.4045459332356646e-08
26
5.526938975085958e-08
27
5.737769354616752e-08
28
6.039495080872834e-08
29
6.438243726019078e-08
30
6.943730995892001e-08
31
7.569407405737872e-08
32
8.332803112753743e-08
33
9.256060885378132e-08
34
1.036666165746281e-07
